In [21]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date

In [22]:
## I acquire stock list in thailand ###
ex = pd.read_excel('listedCompanies_th_TH.xlsx', sheet_name='listedCompanies_th_TH',skiprows=1)
df = pd.DataFrame(ex['หลักทรัพย์'].astype('str') + '.bk')
df = df.rename(columns={'หลักทรัพย์' : 'ticker'})
df = df.reset_index(drop=True)
df = df[:50]

In [23]:
# df = pd.DataFrame(['MO','ASO','HPQ','AMCX','AMLX','CROX','KO'],columns=['ticker'])
# df

### Acquire informations from yfinance function >> .info

In [24]:
# 1st pulling date for this data #
df['date_pulling'] = date.today()

In [25]:
# 2nd acquiring information from .info #
info_attribute_list = [
    'industry',
    'sector',
    'marketCap',
    'currentPrice',
    'enterpriseValue',
]
# 3rd acquiring information from .quarterly_balance_sheet #
balancesheet_list = [
    'Total Assets',
    'Current Liabilities'
]

# 4th acquiring information from .quarterly_financials #
ttm = 4
financials_list = [
    'EBIT'
]

info_attribute_list_buffer = []
balancesheet_list_buffer = []
financials_list_buffer = []
for i in range(len(df)):
    yfticker = yf.Ticker(df['ticker'][i])

    # 2nd acquiring information from .info #
    info_attribute_list_buffer_r = []
    for j in range(len(info_attribute_list)):
        try:
            info_attribute_list_buffer_r.append(yfticker.info[info_attribute_list[j]])
        except:
            info_attribute_list_buffer_r.append(None)
    info_attribute_list_buffer.append(info_attribute_list_buffer_r)

    # 3rd acquiring information from .balancesheet #
    balancesheet_list_buffer_r = []
    for j in range(len(balancesheet_list)):
        try:
            balancesheet_list_buffer_r.append(yfticker.quarterly_balance_sheet.loc[yfticker.quarterly_balance_sheet.index == balancesheet_list[j]].values[0][0])
        except:
            balancesheet_list_buffer_r.append(None)
    balancesheet_list_buffer.append(balancesheet_list_buffer_r)

    # 4th acquiring information from .quarterly_financials #
    financials_list_buffer_r = []
    for j in range(len(financials_list)):
        try:
            financials_list_buffer_r.append(np.array([yfticker.quarterly_financials.loc[yfticker.quarterly_financials.index == financials_list[j]].values[0][i] for i in range(ttm)]).sum())
        except:
            financials_list_buffer_r.append(None)
    try:
        financials_list_buffer_r.append(yfticker.quarterly_financials.columns[0])
    except:
        financials_list_buffer_r.append(None)
    financials_list_buffer.append(financials_list_buffer_r)

financials_list.append('ttm_lastest')
df = df.join(pd.DataFrame(info_attribute_list_buffer, columns=info_attribute_list))
df = df.join(pd.DataFrame(balancesheet_list_buffer, columns=balancesheet_list))
df = df.join(pd.DataFrame(financials_list_buffer, columns=financials_list))


##### Drop NAN #############
df = df.dropna()
##### Drop where marketcap and EBIT < 0#############
df = df.loc[df['enterpriseValue'] >0]
df = df.loc[df['EBIT'] >0]
df = df.reset_index(drop=True)


In [26]:
df['sector'].unique()

array(['Basic Materials', 'Industrials', 'Real Estate',
       'Consumer Defensive', 'Utilities', 'Consumer Cyclical',
       'Communication Services', 'Financial Services', 'Healthcare',
       'Technology'], dtype=object)

In [27]:
### Drop Utilities, Energy, and Financial Services ###
sectortoexclude = ['Utilites','Energy','Financial Services'],
df = df.loc[df['sector'] != 'Utilities']
df = df.reset_index(drop=True)


In [28]:

### Next I perform calculations for MF_ROC and MF_EY ###
df['MF_ROC'] = df['EBIT']/(df['Total Assets'] - df['Current Liabilities'])
df['MF_EY'] = df['EBIT']/df['enterpriseValue']
df

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY
0,2S.bk,2024-08-10,Steel,Basic Materials,1.594989e+09,2.90,1.325299e+09,2.400035e+09,3.039420e+08,1.225020e+08,2024-03-31,0.058443,0.092433
1,3K-BAT.bk,2024-08-10,Electrical Equipment & Parts,Industrials,4.174800e+09,53.25,3.860087e+09,3.736470e+09,8.851644e+08,2.054459e+08,2024-03-31,0.072053,0.053223
2,A5.bk,2024-08-10,Real Estate - Development,Real Estate,3.369080e+09,2.86,4.583295e+09,3.836102e+09,1.114288e+09,6.915935e+08,2024-03-31,0.254093,0.150894
3,AAI.bk,2024-08-10,Packaged Foods,Consumer Defensive,1.253750e+10,5.90,1.170753e+10,5.844300e+09,5.309470e+08,6.072975e+08,2024-03-31,0.114296,0.051872
4,AAV.bk,2024-08-10,Airlines,Industrials,2.904100e+10,2.26,7.162734e+10,7.053564e+10,2.740129e+10,1.790460e+09,2024-03-31,0.041509,0.024997
5,ACG.bk,2024-08-10,Auto & Truck Dealerships,Consumer Cyclical,4.980000e+08,0.83,7.881516e+08,1.373075e+09,4.725849e+08,5.740283e+07,2024-03-31,0.063746,0.072832
6,ADD.bk,2024-08-10,Advertising Agencies,Communication Services,6.368000e+08,3.98,4.239278e+08,6.352905e+08,6.716237e+07,4.025532e+07,2024-03-31,0.070856,0.094958
7,ADVANC.bk,2024-08-10,Telecom Services,Communication Services,7.286300e+11,245.00,9.405999e+11,4.385193e+11,1.311503e+11,4.786043e+10,2024-06-30,0.155710,0.050883
8,AF.bk,2024-08-10,Credit Services,Financial Services,8.960000e+08,0.56,2.557790e+09,2.399439e+09,1.808713e+09,7.934641e+06,2024-03-31,0.013432,0.003102
9,AFC.bk,2024-08-10,Textile Manufacturing,Consumer Cyclical,2.141992e+08,4.70,1.886096e+08,1.477281e+09,1.307217e+08,1.294982e+06,2024-03-31,0.000962,0.006866


In [29]:
### Ranking regarding MFs####

df['Ranking_MF_ROC'] = df['MF_ROC'].rank()
df['Ranking_MF_EY'] = df['MF_EY'].rank()
df['Ranking_MF'] = df['Ranking_MF_ROC'] + df['Ranking_MF_EY']

df.loc[df['Ranking_MF'] == df['Ranking_MF'].max()]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
19,AKR.bk,2024-08-10,Electrical Equipment & Parts,Industrials,1.251115e+09,0.85,1.370824e+09,2.352730e+09,670840499.0,281795015.0,2024-03-31,0.167547,0.205566,31.0,28.0,59.0
20,ALLA.bk,2024-08-10,Industrial Distribution,Industrials,9.300000e+08,1.55,5.452890e+08,1.225429e+09,215087000.0,138185360.0,2024-03-31,0.136771,0.253417,28.0,31.0,59.0


In [30]:
df_sorted = df.sort_values(by=['Ranking_MF'],ascending=False)
df_sorted = df_sorted.reset_index(drop=True)
df_sorted[:50]

,ticker,date_pulling,industry,sector,marketCap,currentPrice,enterpriseValue,Total Assets,Current Liabilities,EBIT,ttm_lastest,MF_ROC,MF_EY,Ranking_MF_ROC,Ranking_MF_EY,Ranking_MF
0,ALLA.bk,2024-08-10,Industrial Distribution,Industrials,9.300000e+08,1.55,5.452890e+08,1.225429e+09,2.150870e+08,1.381854e+08,2024-03-31,0.136771,0.253417,28.0,31.0,59.0
1,AKR.bk,2024-08-10,Electrical Equipment & Parts,Industrials,1.251115e+09,0.85,1.370824e+09,2.352730e+09,6.708405e+08,2.817950e+08,2024-03-31,0.167547,0.205566,31.0,28.0,59.0
2,AMC.bk,2024-08-10,Steel,Basic Materials,1.526705e+09,3.18,1.947399e+09,5.057308e+09,1.531890e+09,4.704230e+08,2024-03-31,0.133438,0.241565,27.0,30.0,57.0
3,A5.bk,2024-08-10,Real Estate - Development,Real Estate,3.369080e+09,2.86,4.583295e+09,3.836102e+09,1.114288e+09,6.915935e+08,2024-03-31,0.254093,0.150894,32.0,24.0,56.0
4,AIT.bk,2024-08-10,Information Technology Services,Technology,6.022767e+09,3.92,3.978415e+09,6.726925e+09,2.212375e+09,7.170632e+08,2024-03-31,0.158834,0.180238,30.0,26.0,56.0
5,AH.bk,2024-08-10,Auto Parts,Consumer Cyclical,5.271983e+09,15.20,9.349102e+09,2.547749e+10,1.105056e+10,1.848404e+09,2024-03-31,0.128122,0.197709,26.0,27.0,53.0
6,AHC.bk,2024-08-10,Medical Care Facilities,Healthcare,2.293608e+09,15.30,1.053337e+09,2.448409e+09,2.518110e+08,2.166742e+08,2024-03-31,0.098641,0.205703,23.0,29.0,52.0
7,ALUCON.bk,2024-08-10,Aluminum,Basic Materials,7.646400e+09,177.00,4.585416e+09,7.890793e+09,5.772030e+08,7.327545e+08,2024-03-31,0.100191,0.159801,24.0,25.0,49.0
8,AMA.bk,2024-08-10,Marine Shipping,Industrials,2.227043e+09,4.30,3.457082e+09,4.946493e+09,8.468544e+08,3.566900e+08,2024-03-31,0.087005,0.103177,21.0,23.0,44.0
9,AKP.bk,2024-08-10,Waste Management,Industrials,2.545200e+08,0.63,1.048640e+08,7.101190e+08,5.133900e+07,3.237126e+07,2024-03-31,0.049138,0.308698,10.0,32.0,42.0
